This is for Week3 Project1--Neighbours in Toronto DataSet

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('All libraries are imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

First Need to scrape this page'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [2]:
url_toronto = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url_toronto)
result.headers

{'Date': 'Tue, 03 Dec 2019 01:22:48 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.5', 'X-ATS-Timestamp': '1575336168', 'Content-Type': 'text/html; charset=UTF-8', 'X-Powered-By': 'PHP/7.2.24-1+0~20191026.31+debian9~1.gbpbbacde+wmf1', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Last-Modified': 'Sun, 17 Nov 2019 21:20:34 GMT', 'Backend-Timing': 'D=114308 t=1575235234084619', 'Content-Encoding': 'gzip', 'X-Varnish': '877059185 665670835', 'Age': '67940', 'X-Cache': 'cp2012 hit, cp2004 hit/17', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=03-Dec-2019;Path=/;HttpOnly;secure;Expires=Sat, 04 Jan 2020 12:00:00 GMT, WMF-Last-Access-Global=03-Dec-2019;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Sat, 04 Jan 2020 12

In [5]:
soup = BeautifulSoup(result.content, 'html.parser')

# Table Design

In [18]:
table= soup.find('table')
all_fields = table.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(all_fields), 3):
    postcode.append(all_fields[i].text.strip())
    borough.append(all_fields[i+1].text.strip())
    neighbourhood.append(all_fields[i+2].text.strip())
        
dataframe= pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
dataframe.columns = ['Postcode', 'Borough', 'Neighbourhood']
print(dataframe.shape)
dataframe.head()

(287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Remove "Not assigned" values

In [19]:
dataframe['Borough'].replace('Not assigned', np.nan, inplace=True)
dataframe.dropna(subset=['Borough'], inplace=True)
print(dataframe.shape)
dataframe.head()

(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [20]:
dataframe2= dataframe.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
dataframe2.columns = ['Postcode', 'Borough', 'Neighbourhood']
print(dataframe2.shape)
dataframe2.head()

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
dataframe2.shape

(103, 3)

In [22]:
geo_data = pd.read_csv('http://cocl.us/Geospatial_data')
geo_data.columns = ['Postcode', 'Latitude', 'Longitude']

In [23]:
last_table= pd.merge(dataframe2, geo_data, on=['Postcode'], how='inner')
toronto_table= last_table[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()
toronto_table.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


# Map


In [35]:
address = 'Toronto'
geolocator = Nominatim()

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(address, latitude, longitude))


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinates of Toronto are Toronto, 43.653963.


In [37]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_table['Latitude'], toronto_table['Longitude'], toronto_table['Borough'], toronto_table['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(toronto_map)  
    
toronto_map